In [2]:
#!/usr/bin/env python
# coding: utf-8

import os
import pandas as pd
import requests
import numpy as np
import json
import login
from datetime import datetime, timedelta
from openpyxl import load_workbook
import os
from geopy.geocoders import Nominatim
# ------------------ #
# Customer DB Params #
# ------------------ #

companyId = 92
userName = "adriano.alvesbitencourtdosanjos@gmail.com"

companyDf = pd.read_excel("company_Adriano.xlsx") #Change directory
company = companyDf[companyDf["companyId"] == companyId]

db_url = company["endPoint"].values[0].replace('/parse/', '')
appId = company["appId"].values[0]

sessionToken = login.login(userName, db_url, appId)

header = {
    "Content-Type": "application/json",
    "X-Parse-Application-Id": appId,
    'X-Parse-Session-Token': sessionToken
}



# ------------ #
# Cooler Query #
# ------------ #
table = "/parse/classes/Cooler"

field_list = [
    "coolerId",
    "usageStatus",
    "customPatrimonio",
    "controllerId",
    "oemSerial",
    # "lastTracked",
    # "customSapId",
    # "customNivelDados",
]

cooler_df = pd.DataFrame() # Will hold cooler data
urlParams = {
    "keys": (",".join(field_list)),
    "limit": "1000000"
}

response = requests.get(db_url + table, headers=header, params=urlParams)
cooler_data_json = response.json()["results"]
cooler_df = pd.DataFrame.from_records(cooler_data_json, exclude=["createdAt", "updatedAt"])

cooler_df.to_excel('koabrfrio.xlsx')

In [2]:
df_aofrio_abr = pd.read_excel('Aofrio_Abr_.xlsx', sheet_name='base')

nome_para_excluir = 'Scrapped'
nome_para_excluir2 = 'Disabled'

# Excluir todas as linhas com os nomes especificados
df = cooler_df.loc[(cooler_df['usageStatus'] != nome_para_excluir) & (cooler_df['usageStatus'] != nome_para_excluir2)].copy()
df.drop_duplicates('controllerId', inplace=True)

# Gerar as colunas que serão utilizadas para fazer as consultas
df['G'] = None
df['H'] = None
df['I'] = None
df['J'] = None
df['K'] = None

# Juntando coolerId e Patrimonio mas o patrimonio tem prioridade
def minha_funcao(row):
    if row['customPatrimonio'] != "-":
        return row['customPatrimonio']
    elif row['coolerId'] in df['customPatrimonio'].values:
        return df.loc[df['customPatrimonio'] == row['coolerId'], 'G'].iloc[0]
    else:
        return row['coolerId']

# Função para pegar os últimos 7 dígitos do coolerId
def aplicar_formula(valor):
    valor_str = str(valor)
    return  valor_str[-7:]

# Função para converter valores para números, se possível
def converter_para_numero(valor):
    try:
        return float(valor)
    except ValueError:
        return valor

# Aplicar a função em uma nova coluna
df['G'] = df.apply(minha_funcao, axis=1)

# Aplicar a fórmula na coluna 'G' e armazenar o resultado na coluna 'H'
df['H'] = df['G'].apply(aplicar_formula)

# Converter os valores da coluna 'H' para números (se possível)
df['H'] = df['H'].apply(converter_para_numero)

sub_df_aofrio = df_aofrio_abr['Nº de série']

df.rename(columns={'G': 'Nº de série'}, inplace=True)

# Realizar a junção dos DataFrames usando a coluna 'Nº de série'
df_merge = pd.merge(df, df_aofrio_abr, how='inner', on='Nº de série')

# Adicionar os resultados correspondentes à coluna 'I'
df['I'] = df['Nº de série'].isin(df_merge['Nº de série']).map({True: 'Match', False: '-'})

df_aofrio_abr.rename(columns={'Nº de série': 'H'}, inplace=True)

df_merge2 = pd.merge(df, df_aofrio_abr, how='inner', on='H')

df['J'] = df['H'].isin(df_merge2['H']).map({True: 'Match', False: '-'})

df


,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial,Nº de série,H,I,J,K
1,TEumrz57VI,ERJ 987582,In service,-,NaN,NaN,ERJ 987582,987582.0,-,-,None
14,T8QV1YuSHx,1132912383025414651,Lost,-,04kD5o7Nn0079092017,1132912383025414651200,1132912383025414651,5414651.0,-,-,None
16,hqookQj8u2,GEMF011431,In service,GEMF011431,101B103380079092017,1230533300037952891,GEMF011431,F011431,Match,-,None
17,zsN3F9KJBN,1231783618038034067,Warehoused,GEMF019535,101B104350079092017,123178361803803406720001,GEMF019535,F019535,Match,-,None
19,xR6mnO3b63,GEMF025329,In service,GEMF025329,101B104460079092017,1232933300038114133,GEMF025329,F025329,Match,-,None
...,...,...,...,...,...,...,...,...,...,...,...
80270,q35hLvXWhA,teste Click,Lost,-,hggA3tmEn0079092017,teste Click,teste Click,e Click,-,-,None
80271,DZ5heB0Yq8,1160602383026095245,Test,-,hilF3tmEn0079092017,116060238302609524520005,1160602383026095245,6095245.0,-,-,None
80273,MQtWcN8dlt,GERJ963487,Lost,-,kgUI5o7Nn0079092017,113176238302531747520007,GERJ963487,J963487,-,-,None
80274,xH2yAQNgqq,GERJ930498,Lost,-,kkeD5o7Nn0079092017,sem serial,GERJ930498,J930498,-,-,None


In [3]:
df_aofrio_abr = pd.read_excel('Aofrio_Abr_.xlsx', sheet_name='base')

nome_para_excluir = 'Scrapped'
nome_para_excluir2 = 'Disabled'

# Excluir todas as linhas com os nomes especificados
df = cooler_df.loc[(cooler_df['usageStatus'] != nome_para_excluir) & (cooler_df['usageStatus'] != nome_para_excluir2)].copy()
df.drop_duplicates('controllerId', inplace=True)

# Gerar as colunas que serão utilizadas para fazer as consultas
df['G'] = None
df['H'] = None
df['I'] = None
df['J'] = None
df['K'] = None

# Juntando coolerId e Patrimonio mas o patrimonio tem prioridade
def minha_funcao(row):
    if row['customPatrimonio'] != "-":
        return row['customPatrimonio']
    elif row['coolerId'] in df['customPatrimonio'].values:
        return df.loc[df['customPatrimonio'] == row['coolerId'], 'G'].iloc[0]
    else:
        return row['coolerId']

# Função para pegar os últimos 7 dígitos do coolerId
def aplicar_formula(valor):
    valor_str = str(valor)
    return  valor_str[-7:]

# Função para converter valores para números, se possível
def converter_para_numero(valor):
    try:
        return float(valor)
    except ValueError:
        return valor

# Aplicar a função em uma nova coluna
df['G'] = df.apply(minha_funcao, axis=1)

# Aplicar a fórmula na coluna 'G' e armazenar o resultado na coluna 'H'
df['H'] = df['G'].apply(aplicar_formula)

# Converter os valores da coluna 'H' para números (se possível)
df['H'] = df['H'].apply(converter_para_numero)

sub_df_aofrio = df_aofrio_abr['Nº de série']

df.rename(columns={'G': 'Nº de série'}, inplace=True)

# Realizar a junção dos DataFrames usando a coluna 'Nº de série'
df_merge = pd.merge(df, df_aofrio_abr, how='inner', on='Nº de série')

# Adicionar os resultados correspondentes à coluna 'I'
df['I'] = df['Nº de série'].isin(df_merge['Nº de série']).map({True: 'Match', False: '-'})

df_aofrio_abr.rename(columns={'Nº de série': 'H'}, inplace=True)

df_merge2 = pd.merge(df, df_aofrio_abr, how='inner', on='H')

df['J'] = df['H'].isin(df_merge2['H']).map({True: 'Match', False: '-'})

#função para combinar as colunas i e j na coluna K para saber qual dar match com a base 
def combinar_match(row):
    if row['I'] == 'Match' or row['J'] == 'Match':
        return 'Match'
    else:
        return '-'

df['K'] = df.apply(combinar_match, axis=1)

df.to_excel('veretidoFinal.xlsx')

In [4]:
df_att = df.copy()

df_att.drop(['I', 'J'], axis=1, inplace=True)

nome_para_excluir = '-'

df_dados_iguais = df_att.loc[(df_att['K'] != nome_para_excluir)].copy()

df_dados_iguais.drop_duplicates('controllerId', inplace=True)

df_dados_iguais

,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial,Nº de série,H,K
16,hqookQj8u2,GEMF011431,In service,GEMF011431,101B103380079092017,1230533300037952891,GEMF011431,F011431,Match
17,zsN3F9KJBN,1231783618038034067,Warehoused,GEMF019535,101B104350079092017,123178361803803406720001,GEMF019535,F019535,Match
19,xR6mnO3b63,GEMF025329,In service,GEMF025329,101B104460079092017,1232933300038114133,GEMF025329,F025329,Match
22,1a6EfAGTTV,GEMF020182,In service,GEMF020182,101B105550079092017,123175330003803187420005,GEMF020182,F020182,Match
24,21ODdoxsYz,GEMF014221,In service,GEMF014221,101B105680079092017,123093330003797650420005,GEMF014221,F014221,Match
...,...,...,...,...,...,...,...,...,...
80247,xvzN0MDcAo,6559381,In service,6559381,F3D355415C079092017,117361094802655938120003,6559381,6559381.0,Match
80248,FECdiJGChz,gerj 983907,In service,GERJ983907,F3D483FBA6079092017,120062535902715267820001,GERJ983907,J983907,Match
80251,P7Lztt9KaP,Gerj975991,In service,GERJ975991,F3D8E1BAF6089092017,NaN,GERJ975991,J975991,Match
80253,zWO7ccp6EE,GERP044518,In service,GERP044518,F3E4BB4AB3079092017,118024094802657748120005,GERP044518,P044518,Match


In [5]:
nome_para_excluir2 = 'Match'

df_dados_diferentes = df_att.loc[(df_att['K'] != nome_para_excluir2)].copy()

df_dados_diferentes.drop_duplicates('controllerId', inplace=True)

df_dados_diferentes

,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial,Nº de série,H,K
1,TEumrz57VI,ERJ 987582,In service,-,NaN,NaN,ERJ 987582,987582.0,-
14,T8QV1YuSHx,1132912383025414651,Lost,-,04kD5o7Nn0079092017,1132912383025414651200,1132912383025414651,5414651.0,-
20,Hyh2zyVux0,1232933300038114135,Production,-,101B105540079092017,123293330003811413520004,1232933300038114135,8114135.0,-
25,pVx2DGWa1b,1233110272028127938,Production,-,101B105950079092017,123311027202812793820008,1233110272028127938,8127938.0,-
26,7yZS5I8jV8,1233110272028127925,Production,-,101B106550079092017,123311027202812792520008,1233110272028127925,8127925.0,-
...,...,...,...,...,...,...,...,...,...
80270,q35hLvXWhA,teste Click,Lost,-,hggA3tmEn0079092017,teste Click,teste Click,e Click,-
80271,DZ5heB0Yq8,1160602383026095245,Test,-,hilF3tmEn0079092017,116060238302609524520005,1160602383026095245,6095245.0,-
80273,MQtWcN8dlt,GERJ963487,Lost,-,kgUI5o7Nn0079092017,113176238302531747520007,GERJ963487,J963487,-
80274,xH2yAQNgqq,GERJ930498,Lost,-,kkeD5o7Nn0079092017,sem serial,GERJ930498,J930498,-


In [91]:
import os
import pandas as pd
import requests
import numpy as np
import json
from datetime import datetime, timedelta
from openpyxl import load_workbook
import os
from geopy.geocoders import Nominatim

resource = "/parse/login"
 
def login(userName, password, serverurl, AppID):
    header = {'Content-Type': 'application/json',
              'X-Parse-Application-Id': AppID} 

    payload = {'username': userName, 
               'password': password}
    response_decoded_json = requests.post(serverurl + resource, json=payload, headers=header)
    response_dict = response_decoded_json.json()
    return(response_dict['sessionToken'])

companyId = int(input("Digite o Numero company_id:"))
userName = input("Digite o email user_name:")
password = input("Digite a senha: ")

companyDf = pd.read_excel("company_Adriano.xlsx")  # Change directory
company = companyDf[companyDf["companyId"] == companyId]

db_url = company["endPoint"].values[0].replace('/parse/', '')
appId = company["appId"].values[0]

sessionToken = login(userName, password, db_url, appId)

header = {
    "Content-Type": "application/json",
    "X-Parse-Application-Id": appId,
    'X-Parse-Session-Token': sessionToken
}

# Cooler Query
table = "/parse/classes/Cooler"
field_list = [
    "coolerId",
    "usageStatus",
    "customPatrimonio",
    "controllerId",
    "oemSerial",
]

cooler_df = pd.DataFrame()  # Will hold cooler data
urlParams = {
    "keys": (",".join(field_list)),
    "limit": "1000000"
}

response = requests.get(db_url + table, headers=header, params=urlParams)
cooler_data_json = response.json()["results"]
cooler_df = pd.DataFrame.from_records(cooler_data_json, exclude=["createdAt", "updatedAt"])

cooler_df


{'objectId': 'iY5gGKPw43', 'outletIds': [], 'username': 'adriano.alvesbitencourtdosanjos@gmail.com', 'email': 'adriano.alvesbitencourtdosanjos@gmail.com', 'fullname': 'Adriano', 'appAuthCode': '09271659544', 'createdBy': {'__type': 'Pointer', 'className': '_User', 'objectId': 'cJtRNOkWiH'}, 'updatedBy': {'__type': 'Pointer', 'className': '_User', 'objectId': 'cJtRNOkWiH'}, 'orgLeft': 100000000, 'orgRight': 199999999, 'bottomLevel': False, 'verificationCode': '092-pY7CJqH45Ccby4NqkjTAysCYHzWQ8kTA6UGHyJBv8KmVaMUuzV', 'createdAt': '2024-05-13T16:13:37.740Z', 'updatedAt': '2024-05-29T13:58:49.445Z', 'appDevices': {'report': {'devices': [{'id': '342D3382', 'model': 'PCware IPMH110Pro', 'osVersion': 'Windows 10', 'username': 'adria', 'appVersion': '4.20.82', 'createdUtc': 1715623449048, 'updatedUtc': 1716991129442}]}}, 'ACL': {'*': {'read': True}, 'iY5gGKPw43': {'read': True, 'write': True}}, 'sessionToken': 'r:dc07ddba77c7a2b4e818fa5399c72034'}


,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial
0,3XGdMD9cdv,GERJ937341,Scrapped,-,NaN,NaN
1,TEumrz57VI,ERJ 987582,In service,-,NaN,NaN
2,6jXUXsxpQG,ERJ988394,In service,-,NaN,NaN
3,BjKIF0O6tn,EMF 006476,In service,GEMF006476,NaN,NaN
4,GBdRXUpQ0y,70220&3296,Scrapped,-,NaN,NaN
...,...,...,...,...,...,...
80432,PFW8iRk0J8,GERP042684,Scrapped,-,iNqA3tmEn0079092017,114332231702574832420002
80433,MQtWcN8dlt,GERJ963487,Lost,-,kgUI5o7Nn0079092017,113176238302531747520007
80434,xH2yAQNgqq,GERJ930498,Lost,-,kkeD5o7Nn0079092017,sem serial
80435,3q0IGBOoZF,GERJ959696,Scrapped,-,qKdI5o7Nn0079092017,1012189978


In [83]:
df_aofrio_abr = pd.read_excel('Andina - Relatório de Embarque 14.05.2024.xlsx', sheet_name='Ativos')

nome_para_excluir = 'Scrapped'
nome_para_excluir2 = 'Disabled'

# Excluir todas as linhas com os nomes especificados
df = cooler_df.loc[(cooler_df['usageStatus'] != nome_para_excluir) & (cooler_df['usageStatus'] != nome_para_excluir2)].copy()
df.drop_duplicates('controllerId', inplace=True)

# Gerar as colunas que serão utilizadas para fazer as consultas
df['G'] = None
df['H'] = None
df['I'] = None
df['J'] = None
df['K'] = None

# Juntando coolerId e Patrimonio mas o patrimonio tem prioridade
def minha_funcao(row):
    if row['customPatrimonio'] != "-":
        return row['customPatrimonio']
    elif row['coolerId'] in df['customPatrimonio'].values:
        return df.loc[df['customPatrimonio'] == row['coolerId'], 'G'].iloc[0]
    else:
        return ''

df['G'] = df.apply(minha_funcao, axis=1)

df_tem_custompatrimonio = df.loc[(df['G'] != '')].copy()

df_tem_custompatrimonio.dropna(subset=['G'], inplace=True)

df_tem_custompatrimonio

,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial,G,H,I,J,K
16,hqookQj8u2,GEMF011431,In service,GEMF011431,101B103380079092017,1230533300037952891,GEMF011431,None,None,None,None
17,zsN3F9KJBN,1231783618038034067,Warehoused,GEMF019535,101B104350079092017,123178361803803406720001,GEMF019535,None,None,None,None
19,xR6mnO3b63,GEMF025329,In service,GEMF025329,101B104460079092017,1232933300038114133,GEMF025329,None,None,None,None
22,1a6EfAGTTV,GEMF020182,In service,GEMF020182,101B105550079092017,123175330003803187420005,GEMF020182,None,None,None,None
24,21ODdoxsYz,GEMF014221,In service,GEMF014221,101B105680079092017,123093330003797650420005,GEMF014221,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
80379,xvzN0MDcAo,6559381,In service,6559381,F3D355415C079092017,117361094802655938120003,6559381,None,None,None,None
80380,FECdiJGChz,gerj 983907,In service,GERJ983907,F3D483FBA6079092017,120062535902715267820001,GERJ983907,None,None,None,None
80383,P7Lztt9KaP,Gerj975991,In service,GERJ975991,F3D8E1BAF6089092017,NaN,GERJ975991,None,None,None,None
80385,zWO7ccp6EE,GERP044518,In service,GERP044518,F3E4BB4AB3079092017,118024094802657748120005,GERP044518,None,None,None,None


In [99]:
def aplicar_formula(valor):
    valor_str = str(valor)
    return  valor_str[-7:]

def converter_para_numero(valor):
    try:
        return float(valor)
    except ValueError:
        return valor

df_nao_tem_custompatrimonio = df.loc[(df['G'] == '' )].copy()

df_nao_tem_custompatrimonio['G'] = df_nao_tem_custompatrimonio['coolerId'].apply(aplicar_formula)

df_nao_tem_custompatrimonio['G'] = df_nao_tem_custompatrimonio['G'].apply(converter_para_numero)

df_nao_tem_custompatrimonio.rename(columns={'G': 'Nr. Série'}, inplace=True)

sub_df_aofrio = df_aofrio_abr[['Nr. Série', '1º Patrimonial', '2º Patrimonial']]

sub_df_aofrio['Nr. Série'] = sub_df_aofrio['Nr. Série'].apply(converter_para_numero)

df_nao_tem_custompatrimonio['Nr. Série'] = df_nao_tem_custompatrimonio['Nr. Série'].astype(str)

sub_df_aofrio['Nr. Série'] = sub_df_aofrio['Nr. Série'].astype(str)

# Realizar a junção dos DataFrames usando a coluna 'Nº de série'
df_merge = pd.merge(df_nao_tem_custompatrimonio, sub_df_aofrio, how='inner', on='Nr. Série')

df_merge.drop(['I', 'J', 'K','H'], axis=1, inplace=True)

df_concatenado2 = pd.concat([df_merge, df_tem_custompatrimonio], ignore_index=True)

df_concatenado2.drop(['I', 'J', 'K','H', 'G'], axis=1, inplace=True)

df_concatenado2.drop_duplicates('coolerId', inplace=True)

df_concatenado = pd.concat([df_merge, df_nao_tem_custompatrimonio], ignore_index=True)

# Remover linhas duplicadas pelo 'coolerId' e manter apenas aquelas em que a coluna '1º Patrimonial' não é NaN
df_sem_duplicatas = df_concatenado.drop_duplicates(subset='coolerId')

df_nan = df_sem_duplicatas[df_sem_duplicatas['1º Patrimonial'].isna()]

df_nan_att = df_nan.copy()

df_nan_att.drop(['I', 'J', 'K','H', '2º Patrimonial','1º Patrimonial', 'Nr. Série'], axis=1, inplace=True)

df_desco_patri = pd.merge(df_nan_att, sub_df_aofrio, how='left', left_on='coolerId', right_on='1º Patrimonial')

df_com_patri = df_desco_patri[df_desco_patri['1º Patrimonial'].notna()]

df_concatenado3 = pd.concat([df_com_patri, df_concatenado2], ignore_index=True)

def funcao_att(row):
    if row['customPatrimonio'] != "-":
        return row['customPatrimonio']
    elif row['customPatrimonio'] == "-":
        return row['1º Patrimonial']
    else:
        return ''

df_patri_att = df_concatenado3.copy()  

df_patri_att['customPatrimonioAtt'] = None

df_patri_att['customPatrimonioAtt'] = df_patri_att.apply(funcao_att, axis=1)  

df_patri_att.to_excel('customPatrimonioEncontrada.xlsx')

df_sem_patri = df_desco_patri[df_desco_patri['1º Patrimonial'].isna()]

df_sem_patri.to_excel('customPatrimonioNãoEncontrada.xlsx')


C:\Users\adria\AppData\Local\Temp\ipykernel_4524\3952355176.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df_aofrio['Nr. Série'] = sub_df_aofrio['Nr. Série'].apply(converter_para_numero)
C:\Users\adria\AppData\Local\Temp\ipykernel_4524\3952355176.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df_aofrio['Nr. Série'] = sub_df_aofrio['Nr. Série'].astype(str)


,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial,Nr. Série,1º Patrimonial,2º Patrimonial,customPatrimonioAtt
0,0h9nfthHMC,GEMF027134,In service,-,138A020970079092017,034002812967120001,8129671.0,GEMF027134,NaN,GEMF027134
1,CJKzO9CCcr,GEMF026525,In service,-,139A013220079092017,1233110272028127941,8127941.0,GEMF026525,NaN,GEMF026525
2,F1WGv4lHYV,GEMF027772,In service,-,144A005670079092017,1233110272028128009,8128009.0,GEMF027772,NaN,GEMF027772
3,I8RymEsIh5,GEMF027338,In service,-,144A006740079092017,1233110272028128016,8128021.0,GEMF027338,NaN,GEMF027338
4,pyS6n9ChAX,GEMF027178,In service,-,7022001055079092017,1233110272028127971,8127971.0,GEMF027178,NaN,GEMF027178
...,...,...,...,...,...,...,...,...,...,...
49129,xvzN0MDcAo,6559381,In service,6559381,F3D355415C079092017,117361094802655938120003,NaN,NaN,NaN,6559381
49130,FECdiJGChz,gerj 983907,In service,GERJ983907,F3D483FBA6079092017,120062535902715267820001,NaN,NaN,NaN,GERJ983907
49131,P7Lztt9KaP,Gerj975991,In service,GERJ975991,F3D8E1BAF6089092017,NaN,NaN,NaN,NaN,GERJ975991
49132,zWO7ccp6EE,GERP044518,In service,GERP044518,F3E4BB4AB3079092017,118024094802657748120005,NaN,NaN,NaN,GERP044518


In [73]:
# Concatenar os DataFrames df_merge e df_nao_tem_custompatrimonio
df_concatenado = pd.concat([df_merge, df_nao_tem_custompatrimonio], ignore_index=True)

# Remover linhas duplicadas pelo 'coolerId' e manter apenas aquelas em que a coluna '1º Patrimonial' não é NaN
df_sem_duplicatas = df_concatenado.drop_duplicates(subset='coolerId')

df_nan = df_sem_duplicatas[df_sem_duplicatas['1º Patrimonial'].isna()]

df_nan_att = df_nan.copy()

df_nan_att.drop(['I', 'J', 'K','H', '2º Patrimonial','1º Patrimonial', 'Nr. Série'], axis=1, inplace=True)

df_desco_patri = pd.merge(df_nan_att, sub_df_aofrio, how='left', left_on='coolerId', right_on='1º Patrimonial')

df_com_patri = df_desco_patri[df_desco_patri['1º Patrimonial'].notna()]



In [86]:
# Realizar a junção dos DataFrames usando as colunas 'Nr. Série' e '1º Patrimonial'
df_desco_patri = pd.merge(df_nan_att, sub_df_aofrio, how='left', left_on='coolerId', right_on='1º Patrimonial')

df_com_patri = df_desco_patri[df_desco_patri['1º Patrimonial'].notna()]

#df_com_patri
df

,objectId,coolerId,usageStatus,customPatrimonio,controllerId,oemSerial,G,H,I,J,K
1,TEumrz57VI,ERJ 987582,In service,-,NaN,NaN,,None,None,None,None
14,T8QV1YuSHx,1132912383025414651,Lost,-,04kD5o7Nn0079092017,1132912383025414651200,,None,None,None,None
16,hqookQj8u2,GEMF011431,In service,GEMF011431,101B103380079092017,1230533300037952891,GEMF011431,None,None,None,None
17,zsN3F9KJBN,1231783618038034067,Warehoused,GEMF019535,101B104350079092017,123178361803803406720001,GEMF019535,None,None,None,None
19,xR6mnO3b63,GEMF025329,In service,GEMF025329,101B104460079092017,1232933300038114133,GEMF025329,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
80402,q35hLvXWhA,teste Click,Lost,-,hggA3tmEn0079092017,teste Click,,None,None,None,None
80403,DZ5heB0Yq8,1160602383026095245,Test,-,hilF3tmEn0079092017,116060238302609524520005,,None,None,None,None
80405,MQtWcN8dlt,GERJ963487,Lost,-,kgUI5o7Nn0079092017,113176238302531747520007,,None,None,None,None
80406,xH2yAQNgqq,GERJ930498,Lost,-,kkeD5o7Nn0079092017,sem serial,,None,None,None,None


In [8]:
import re

dftrolado = pd.read_excel('customPatrimonioNãoEncontrada.xlsx')
dfaofrio = pd.read_excel('Andina - Relatório de Embarque 14.05.2024.xlsx', sheet_name='Ativos')

subdfaofrio = dfaofrio[['Nr. Série', '1º Patrimonial']]

subdfteste = dftrolado.copy()

# Garantir que a coluna 'coolerId' seja do tipo string
subdfteste['coolerId'] = subdfteste['coolerId'].astype(str)

def funcao_teste(row):
    pattern = r'^[A-Za-z]{4}\d{6}$'
    if re.match(pattern, row['coolerId']):
        return row['coolerId']
    elif len(row['coolerId']) == 19:
        return row['coolerId']
    else:
        return np.nan

subdfteste.drop(['2º Patrimonial','1º Patrimonial', 'Nr. Série'], axis=1, inplace=True)

subdfteste['H'] = None

subdfteste['H'] = subdfteste.apply(funcao_teste, axis=1)

dfsemimpostor = subdfteste[subdfteste['H'].notna()]

dfimpostor = subdfteste[subdfteste['H'].isna()]



#dfimpostor.to_excel('impostores.xlsx')
#dfsemimpostor.to_excel('certos.xlsx')


In [4]:
import pandas as pd

df_sem_matching = pd.read_excel('Sem_matching.xlsx')
df_sub_semmatch = pd.read_excel('Aofrio_Abr_.xlsx', sheet_name='base')

In [8]:
def aplicar_formula(valor):
    valor_str = str(valor)
    return valor_str[:10]

def aplicar_formula2(valor):
    valor_str = str(valor)
    return valor_str[:14]

def aplicar_formulaB(valor):
    valor_str = str(valor)
    return 'B'+valor_str[:9]

def aplicar_formulaB10(valor):
    valor_str = str(valor)
    return 'B'+valor_str[:10]

def aplicar_formulaC(valor):
    valor_str = str(valor)
    return 'C'+valor_str[:10]

def aplicar_formulaN(valor):
    valor_str = str(valor)
    return 'N'+valor_str[:10]

def aplicar_formulaC10(valor):
    valor_str = str(valor)
    return 'C'+valor_str[:9]

def converter_para_numero(valor):
    try:
        return float(valor)
    except ValueError:
        return valor

def combinar_matching(row):
    if row['MatchPorSlice2'] == 'Match' or row['MatchPorControllerIdB'] == 'Match'or row['MatchPorSlice'] == 'Match' or row['MatchPorControllerIdN'] == 'Match' or row['MatchPorControllerIdC'] == 'Match' or row['MatchPorControllerIdC10'] == 'Match' or row['MatchPorControllerIdB10'] == 'Match':
        return 'Match'
    else:
        return '-'


dfsubandina = df_sub_semmatch[['Nº de série', 'Nº série fabricante', 'N º controlador SAP']]

df_sem_matching['slice'] = None
df_sem_matching['MatchPorSlice'] = None
df_sem_matching['slice2'] = None
df_sem_matching['MatchPorSlice2'] = None
df_sem_matching['MatchPorControllerIdB'] = None
df_sem_matching['sliceControllerIdB'] = None
df_sem_matching['MatchPorControllerIdC'] = None
df_sem_matching['sliceControllerIdC'] = None
df_sem_matching['sliceControllerIdC10'] = None
df_sem_matching['MatchPorControllerIdC10'] = None
df_sem_matching['sliceControllerIdB10'] = None
df_sem_matching['MatchPorControllerIdB10'] = None
df_sem_matching['sliceControllerIdN'] = None
df_sem_matching['MatchPorControllerIdN'] = None
df_sem_matching['TodosOsMatch'] = None

df_sem_matching['slice'] = df_sem_matching['controllerId'].apply(aplicar_formula)
df_sem_matching['slice'] = df_sem_matching['slice'].apply(converter_para_numero)
df_sem_matching['slice'] = df_sem_matching['slice'].astype(str)
df_sem_matching['slice2'] = df_sem_matching['oemSerial'].apply(aplicar_formula2)
df_sem_matching['sliceControllerIdB'] = df_sem_matching['controllerId'].apply(aplicar_formulaB)
df_sem_matching['sliceControllerIdN'] = df_sem_matching['controllerId'].apply(aplicar_formulaN)
df_sem_matching['sliceControllerIdC'] = df_sem_matching['controllerId'].apply(aplicar_formulaC)
df_sem_matching['sliceControllerIdC10'] = df_sem_matching['controllerId'].apply(aplicar_formulaC10)
df_sem_matching['sliceControllerIdB10'] = df_sem_matching['controllerId'].apply(aplicar_formulaB10)

merge_por_slice = pd.merge(df_sem_matching, dfsubandina, how='inner', left_on='slice', right_on='Nº série fabricante')

# Adicionar os resultados correspondentes à coluna 'MatchPorSlice'
df_sem_matching['MatchPorSlice'] = np.where(df_sem_matching['slice'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')
df_sem_matching['MatchPorControllerIdB'] = np.where(df_sem_matching['sliceControllerIdB'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')
df_sem_matching['MatchPorControllerIdC'] = np.where(df_sem_matching['sliceControllerIdC'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')
df_sem_matching['MatchPorControllerIdC10'] = np.where(df_sem_matching['sliceControllerIdC10'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')
df_sem_matching['MatchPorControllerIdB10'] = np.where(df_sem_matching['sliceControllerIdB10'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')
df_sem_matching['MatchPorControllerIdN'] = np.where(df_sem_matching['sliceControllerIdN'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')
df_sem_matching['MatchPorSlice2'] = np.where(df_sem_matching['slice2'].isin(dfsubandina['N º controlador SAP']), 'Match', '-')

df_sem_matching['TodosOsMatch'] = df_sem_matching.apply(combinar_matching, axis=1)

df_sem_matching.drop(['sliceControllerIdB', 'sliceControllerIdC', 'sliceControllerIdN', 'sliceControllerIdC10', 'sliceControllerIdB10', 'slice', 'slice2'], axis=1, inplace=True)

df_sem_matching.to_excel('Match_dos_sem_matching.xlsx')